#Trabalho de Mineração de dados não estruturados 

Aluno - Gabriel Souto Ferrante (12602303)

O trabalho compara os seguintes algoritmos para a classificação animal:

###Inception (GoogLeNet) (pré-treinada no ImageNet)
###ResNET-50 (pré-treinada no ImageNet)
###VGGnet-16 (pré-treinada no ImageNet)

É utilizado 150 imagens das 10 classes de animais do COCO Dataset, que estão em diversos cenários. As classes são:

'bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe'

O treinamento é feito com 50 épocas e avaliado a acurácia e precisão alcançada.




# Download da base de dados

In [1]:
#API do COCO DATASET
!pip install pycocotools

#BAIXANDO AS ANOTATIONS DOS DADOS DE TREINO DA BASE DE 2017
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip /content/annotations_trainval2017.zip

--2021-10-16 17:00:23--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.138.241
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.138.241|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  77.3MB/s    in 3.4s    

2021-10-16 17:00:26 (71.6 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  /content/annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


In [2]:

from pycocotools.coco import COCO
import requests
import os
import time
import cv2
from os import listdir
from os.path import isfile, join


def createDirectory(name, animal):
    import os

    path = name
    access_rights = 0o755
    try:
        os.mkdir(path, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % path)
        
    else:
        print ("Successfully created the directory %s " % path)

    os.chdir(name)

    #Criando o diretório da classe
    path = animal
    access_rights = 0o755
    try:
        os.mkdir(path, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % path)
        
    else:
        print ("Successfully created the directory %s " % path)

    os.chdir("..")

  
def downloadAnimal(animal, coco):

    createDirectory('train/', animal)
    cats = coco.loadCats(coco.getCatIds())
    nms=[cat['name'] for cat in cats]

    catIds = coco.getCatIds(catNms=[animal])

    imgIds = coco.getImgIds(catIds=catIds )
    images = coco.loadImgs(imgIds)

    i = 1
    for im in images:
        if i <= 105:
          if not os.path.isfile('train/'+  animal + '/'+ animal+'_'+im['file_name']):
            img_data = requests.get(im['coco_url']).content
            with open('train/'+  animal + '/'+ animal+'_'+im['file_name'], 'wb') as handler:
                handler.write(img_data)
            
            #time.sleep(3)    
          i = i + 1
        else:
            break

def downloadValid(animal, coco):
    createDirectory('test/', animal)
    cats = coco.loadCats(coco.getCatIds())
    nms=[cat['name'] for cat in cats]

    catIds = coco.getCatIds(catNms=[animal])
    imgIds = coco.getImgIds(catIds=catIds )
    images = coco.loadImgs(imgIds)

    i = 1
    for im in images:
        if i <= 45:
          if not os.path.isfile('test/'+ animal + '/'+ animal+'_'+im['file_name']):
            img_data = requests.get(im['coco_url']).content
            with open('test/'+ animal + '/'+ animal+'_'+im['file_name'], 'wb') as handler:
              handler.write(img_data)

            #time.sleep(3)    
          i = i + 1
        else:
            break

In [3]:
animals = [
           'bird',
           'cat',
           'dog',
           'horse',
           'sheep',
           'cow',
           'elephant',
           'bear',
           'zebra',
           'giraffe'
]
coco = COCO('annotations/instances_train2017.json')
cocoValid = COCO('annotations/instances_val2017.json')
for item in animals:
  
  downloadAnimal(item, coco)
  downloadValid(item, cocoValid)


loading annotations into memory...
Done (t=17.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
Successfully created the directory train/ 
Successfully created the directory bird 
Successfully created the directory test/ 
Successfully created the directory bird 
Creation of the directory train/ failed
Successfully created the directory cat 
Creation of the directory test/ failed
Successfully created the directory cat 
Creation of the directory train/ failed
Successfully created the directory dog 
Creation of the directory test/ failed
Successfully created the directory dog 
Creation of the directory train/ failed
Successfully created the directory horse 
Creation of the directory test/ failed
Successfully created the directory horse 
Creation of the directory train/ failed
Successfully created the directory sheep 
Creation of the directory test/ failed
Successfully created the directory sheep 
Creation of the direct

#Usando o Inception

In [4]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 19 kB/s 
     |████████████████████████████████| 6.8 MB 11.8 MB/s 
     |████████████████████████████████| 981 kB 32.9 MB/s 
     |████████████████████████████████| 263 kB 44.2 MB/s 
     |████████████████████████████████| 2.8 MB 45.8 MB/s 
     |████████████████████████████████| 1.2 MB 31.3 MB/s 
     |████████████████████████████████| 287 kB 34.4 MB/s 
     |████████████████████████████████| 468 kB 42.4 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 35.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.7 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 125 kB 43.7 MB/s 
     |████████████████████████████████| 1.3 MB 27.1 MB/s 
     |████████████████████████████████| 243 kB 45.4 MB/s 
     |████████████████████████████████| 271 kB 47.7 MB/s 
     |████████████████████████████████| 160 kB 51.5 MB/s 
  Created wheel f

In [5]:
!mkdir data
!mv train data
!mv test data

In [6]:
#Caso queira baixar o .zip das imagens
#Fiz isso após baixar as imagens usando a runtime com CPU e depois fiz o upload do
#zip com o runtime para GPU
!zip -r data.zip data/

  adding: data/ (stored 0%)
  adding: data/train/ (stored 0%)
  adding: data/train/bear/ (stored 0%)
  adding: data/train/bear/bear_000000292982.jpg (deflated 0%)
  adding: data/train/bear/bear_000000401518.jpg (deflated 0%)
  adding: data/train/bear/bear_000000182316.jpg (deflated 0%)
  adding: data/train/bear/bear_000000319685.jpg (deflated 0%)
  adding: data/train/bear/bear_000000110606.jpg (deflated 0%)
  adding: data/train/bear/bear_000000122936.jpg (deflated 0%)
  adding: data/train/bear/bear_000000264365.jpg (deflated 1%)
  adding: data/train/bear/bear_000000204806.jpg (deflated 0%)
  adding: data/train/bear/bear_000000421986.jpg (deflated 0%)
  adding: data/train/bear/bear_000000213141.jpg (deflated 0%)
  adding: data/train/bear/bear_000000106585.jpg (deflated 0%)
  adding: data/train/bear/bear_000000075791.jpg (deflated 0%)
  adding: data/train/bear/bear_000000479331.jpg (deflated 0%)
  adding: data/train/bear/bear_000000346295.jpg (deflated 0%)
  adding: data/train/bear/bear_

In [ ]:
!unzip data.zip

In [7]:
#Separando dados de treino e validação

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ['DISABLE_V2_BEHAVIOR'] = '1'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ktrain
from ktrain import vision as vis

(train_data, val_data, preproc) = vis.images_from_folder('data/')

Instructions for updating:
non-resource variables are not supported in the long term
Using DISABLE_V2_BEHAVIOR with TensorFlow
Found 1050 images belonging to 10 classes.
Found 450 images belonging to 10 classes.


In [8]:
modelG = vis.image_classifier('pretrained_inception', train_data, val_data)

The normalization scheme has been changed for use with a pretrained_inception model. If you decide to use a different model, please reload your dataset with a ktrain.vision.data.images_from* function.

Is Multi-Label? False
Is Regression? False
87924736/87910968 [==============================] - 1s 0us/step
pretrained_inception model created.


In [9]:
learnerG = ktrain.get_learner(model=modelG, train_data=train_data, val_data=val_data, 
                             workers=8, use_multiprocessing=False, batch_size=64)

In [10]:
learnerG.fit_onecycle(1e-4, 50) #50 épocas



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/50
17/17 [==============================] - ETA: 0s - batch: 8.0000 - size: 61.7647 - loss: 2.9351 - acc: 0.1105

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


17/17 [==============================] - 62s 2s/step - batch: 8.0000 - size: 61.7647 - loss: 2.9353 - acc: 0.1105 - val_loss: 2.8469 - val_acc: 0.1400
Epoch 2/50
17/17 [==============================] - 15s 939ms/step - batch: 8.0000 - size: 61.7647 - loss: 2.6775 - acc: 0.1610 - val_loss: 2.4736 - val_acc: 0.2378
Epoch 3/50
17/17 [==============================] - 16s 976ms/step - batch: 8.0000 - size: 61.7647 - loss: 2.3903 - acc: 0.2267 - val_loss: 2.0512 - val_acc: 0.3489
Epoch 4/50
17/17 [==============================] - 16s 959ms/step - batch: 8.0000 - size: 61.7647 - loss: 2.1812 - acc: 0.2714 - val_loss: 1.7438 - val_acc: 0.4489
Epoch 5/50
17/17 [==============================] - 16s 975ms/step - batch: 8.0000 - size: 61.7647 - loss: 1.9946 - acc: 0.3371 - val_loss: 1.5291 - val_acc: 0.5044
Epoch 6/50
17/17 [==============================] - 16s 981ms/step - batch: 8.0000 - size: 61.7647 - loss: 1.7358 - acc: 0.4162 - val_loss: 1.3907 - val_acc: 0.5467
Epoch 7/50
17/17 [======

In [11]:
predictorG = ktrain.get_predictor(learnerG.model, preproc)

In [12]:
#Fazendo a predição
real = []
predicoes = []
def show_prediction(fname, animal):
    
    predicted = predictorG.predict_filename(fname)[0]
    print('Predição: %s | A classe real: %s' % (predicted, fname))
    predicoes.append(predicted)
    real.append(animal)



In [13]:
def validationG(animal, coco):

    catIds = coco.getCatIds(catNms=[animal])
    imgIds = coco.getImgIds(catIds=catIds )
    images = coco.loadImgs(imgIds)

    for im in images:
          if os.path.isfile('data/test/'+ animal + '/'+ animal+'_'+im['file_name']):
            fname = 'data/test/'+ animal + '/'+ animal+'_'+im['file_name']
            show_prediction(fname, animal)
          

In [14]:
dicio = {}
animals = [
           'bird',
           'cat',
           'dog',
           'horse',
           'sheep',
           'cow',
           'elephant',
           'bear',
           'zebra',
           'giraffe'
]
cocoValid = COCO('annotations/instances_val2017.json')
for item in animals:
  validationG(item, cocoValid)
  dicio[item] = real
  dicio[item+'_predicoes'] = predicoes
  real=[]
  predicoes=[]

loading annotations into memory...
Done (t=0.92s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Predição: cow | A classe real: data/test/bird/bird_000000055299.jpg
Predição: cat | A classe real: data/test/bird/bird_000000523782.jpg
Predição: cat | A classe real: data/test/bird/bird_000000222235.jpg
Predição: bird | A classe real: data/test/bird/bird_000000523811.jpg
Predição: bird | A classe real: data/test/bird/bird_000000147498.jpg
Predição: dog | A classe real: data/test/bird/bird_000000032811.jpg
Predição: dog | A classe real: data/test/bird/bird_000000544811.jpg
Predição: giraffe | A classe real: data/test/bird/bird_000000122927.jpg
Predição: cow | A classe real: data/test/bird/bird_000000159791.jpg
Predição: bear | A classe real: data/test/bird/bird_000000186422.jpg
Predição: cat | A classe real: data/test/bird/bird_000000542776.jpg
Predição: bird | A classe real: data/test/bird/bird_000000186938.jpg
Predição: giraffe | A classe real: data/test/bird/bird_000000277051.jpg
Predição: giraffe | A classe real: data/test/bird/bird_000000337987.jpg
Predição: cow | A classe real: d

In [15]:
#Acurácia e Precisão (+ Matrix de confusão)

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

def calculaAcuraciaPClasse(animal ,real, predicoes):
  #Calcula a acurácia com o que a saida verdadeira, com a predição
  acc = accuracy_score(real, predicoes)
  print(f'Acurácia para a classe {animal} = {acc}')
  

def calculaf1PClasse(animal ,real, predicoes):
  f1 = f1_score(real, predicoes)
  print(f'F1-Score para a classe {animal} = {f1}')

def matrixConfusao(animal, real, predicoes):
  #Atribuindo os verdadeiros negativos, falsos positivos, falsos negativos,
  #e verdadeiros positivos
  tn, fp, fn, tp = confusion_matrix(real, predicoes).ravel()

  print('TP = ',tp)
  print('FP = ',fp)
  print('TN = ',tn)
  print('FN = ',fn) 


In [16]:
i = 0
RealGeral = []
PredicoesGeral = []
for item in dicio:
  real=[]
  predicoes=[]
  i = 0

  while i < 45:
    
    if not '_' in str(item):
      RealGeral.append(1)
      real.append(1)
      if dicio[item][i] == dicio[item+'_predicoes'][i]:
        predicoes.append(1)
        PredicoesGeral.append(1)
      else:
        predicoes.append(0)
        PredicoesGeral.append(0)
    i = i + 1
  if not '_' in str(item):
    calculaAcuraciaPClasse(item, real, predicoes)
    calculaf1PClasse(item, real, predicoes)
    matrixConfusao(item, real, predicoes)

calculaAcuraciaPClasse('GERAL', RealGeral,PredicoesGeral )
calculaf1PClasse('GERAL', RealGeral,PredicoesGeral )
  

Acurácia para a classe bird = 0.3111111111111111
F1-Score para a classe bird = 0.4745762711864407
TP =  14
FP =  0
TN =  0
FN =  31
Acurácia para a classe cat = 0.8
F1-Score para a classe cat = 0.888888888888889
TP =  36
FP =  0
TN =  0
FN =  9
Acurácia para a classe dog = 0.5777777777777777
F1-Score para a classe dog = 0.732394366197183
TP =  26
FP =  0
TN =  0
FN =  19
Acurácia para a classe horse = 0.6444444444444445
F1-Score para a classe horse = 0.7837837837837839
TP =  29
FP =  0
TN =  0
FN =  16
Acurácia para a classe sheep = 0.7333333333333333
F1-Score para a classe sheep = 0.846153846153846
TP =  33
FP =  0
TN =  0
FN =  12
Acurácia para a classe cow = 0.5333333333333333
F1-Score para a classe cow = 0.6956521739130436
TP =  24
FP =  0
TN =  0
FN =  21
Acurácia para a classe elephant = 0.8666666666666667
F1-Score para a classe elephant = 0.9285714285714286
TP =  39
FP =  0
TN =  0
FN =  6
Acurácia para a classe bear = 0.7555555555555555
F1-Score para a classe bear = 0.860759493

#Usando ResNet-50

In [17]:
modelRes = vis.image_classifier('pretrained_resnet50', train_data, val_data)

The normalization scheme has been changed for use with a pretrained_resnet50 model. If you decide to use a different model, please reload your dataset with a ktrain.vision.data.images_from* function.

Is Multi-Label? False
Is Regression? False
94781440/94765736 [==============================] - 1s 0us/step
pretrained_resnet50 model created.


In [18]:
learnerRes = ktrain.get_learner(model=modelRes, train_data=train_data, val_data=val_data, 
                             workers=8, use_multiprocessing=False, batch_size=64)

In [19]:
learnerRes.fit_onecycle(1e-4, 50)



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/50
17/17 [==============================] - ETA: 0s - batch: 8.0000 - size: 61.7647 - loss: 5.4819 - acc: 0.1010

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


17/17 [==============================] - 32s 1s/step - batch: 8.0000 - size: 61.7647 - loss: 5.4823 - acc: 0.1010 - val_loss: 3.6711 - val_acc: 0.1400
Epoch 2/50
17/17 [==============================] - 18s 1s/step - batch: 8.0000 - size: 61.7647 - loss: 4.2582 - acc: 0.1762 - val_loss: 2.9517 - val_acc: 0.2422
Epoch 3/50
17/17 [==============================] - 18s 1s/step - batch: 8.0000 - size: 61.7647 - loss: 3.2038 - acc: 0.3029 - val_loss: 2.4716 - val_acc: 0.3467
Epoch 4/50
17/17 [==============================] - 18s 1s/step - batch: 8.0000 - size: 61.7647 - loss: 2.4535 - acc: 0.4219 - val_loss: 2.1809 - val_acc: 0.4200
Epoch 5/50
17/17 [==============================] - 18s 1s/step - batch: 8.0000 - size: 61.7647 - loss: 1.8083 - acc: 0.5286 - val_loss: 2.0199 - val_acc: 0.4778
Epoch 6/50
17/17 [==============================] - 18s 1s/step - batch: 8.0000 - size: 61.7647 - loss: 1.4562 - acc: 0.5886 - val_loss: 1.9450 - val_acc: 0.5089
Epoch 7/50
17/17 [=====================

In [20]:
predictorRes = ktrain.get_predictor(learnerRes.model, preproc)

In [21]:
#Fazendo a predição
real = []
predicoes = []
def show_prediction(fname, animal):
    
    predicted = predictorRes.predict_filename(fname)[0]
    print('Predição: %s | A classe real: %s' % (predicted, fname))
    predicoes.append(predicted)
    real.append(animal)

In [22]:
def validationRes(animal, coco):

    catIds = coco.getCatIds(catNms=[animal])
    imgIds = coco.getImgIds(catIds=catIds )
    images = coco.loadImgs(imgIds)

    for im in images:
          if os.path.isfile('data/test/'+ animal + '/'+ animal+'_'+im['file_name']):
            fname = 'data/test/'+ animal + '/'+ animal+'_'+im['file_name']
            show_prediction(fname, animal)

In [23]:
dicioRes = {}
animals = [
           'bird',
           'cat',
           'dog',
           'horse',
           'sheep',
           'cow',
           'elephant',
           'bear',
           'zebra',
           'giraffe'
]
cocoValid = COCO('annotations/instances_val2017.json')
for item in animals:
  validationRes(item, cocoValid)
  dicioRes[item] = real
  dicioRes[item+'_predicoes'] = predicoes
  real=[]
  predicoes=[]

loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Predição: sheep | A classe real: data/test/bird/bird_000000055299.jpg
Predição: dog | A classe real: data/test/bird/bird_000000523782.jpg
Predição: giraffe | A classe real: data/test/bird/bird_000000222235.jpg
Predição: bird | A classe real: data/test/bird/bird_000000523811.jpg
Predição: sheep | A classe real: data/test/bird/bird_000000147498.jpg
Predição: dog | A classe real: data/test/bird/bird_000000032811.jpg
Predição: cow | A classe real: data/test/bird/bird_000000544811.jpg
Predição: giraffe | A classe real: data/test/bird/bird_000000122927.jpg
Predição: cat | A classe real: data/test/bird/bird_000000159791.jpg
Predição: bear | A classe real: data/test/bird/bird_000000186422.jpg
Predição: dog | A classe real: data/test/bird/bird_000000542776.jpg
Predição: bird | A classe real: data/test/bird/bird_000000186938.jpg
Predição: bird | A classe real: data/test/bird/bird_000000277051.jpg
Predição: bear | A classe real: data/test/bird/bird_000000337987.jpg
Predição: cow | A classe real: 

In [24]:
i = 0
RealGeral = []
PredicoesGeral = []
for item in dicioRes:
  real=[]
  predicoes=[]
  i = 0

  while i < 45:
    
    if not '_' in str(item):
      real.append(1)
      RealGeral.append(1)
      if dicioRes[item][i] == dicioRes[item+'_predicoes'][i]:
        predicoes.append(1)
        PredicoesGeral.append(1)
      else:
        predicoes.append(0)
        PredicoesGeral.append(0)
    i = i + 1
  if not '_' in str(item):
    calculaAcuraciaPClasse(item, real, predicoes)
    calculaf1PClasse(item, real, predicoes)
    matrixConfusao(item, real, predicoes)

calculaAcuraciaPClasse('GERAL', RealGeral,PredicoesGeral )
calculaf1PClasse('GERAL', RealGeral,PredicoesGeral )

Acurácia para a classe bird = 0.28888888888888886
F1-Score para a classe bird = 0.4482758620689655
TP =  13
FP =  0
TN =  0
FN =  32
Acurácia para a classe cat = 0.5777777777777777
F1-Score para a classe cat = 0.732394366197183
TP =  26
FP =  0
TN =  0
FN =  19
Acurácia para a classe dog = 0.28888888888888886
F1-Score para a classe dog = 0.4482758620689655
TP =  13
FP =  0
TN =  0
FN =  32
Acurácia para a classe horse = 0.5555555555555556
F1-Score para a classe horse = 0.7142857142857143
TP =  25
FP =  0
TN =  0
FN =  20
Acurácia para a classe sheep = 0.7333333333333333
F1-Score para a classe sheep = 0.846153846153846
TP =  33
FP =  0
TN =  0
FN =  12
Acurácia para a classe cow = 0.5555555555555556
F1-Score para a classe cow = 0.7142857142857143
TP =  25
FP =  0
TN =  0
FN =  20
Acurácia para a classe elephant = 0.8444444444444444
F1-Score para a classe elephant = 0.9156626506024096
TP =  38
FP =  0
TN =  0
FN =  7
Acurácia para a classe bear = 0.8
F1-Score para a classe bear = 0.88888

#Usando VGGNet-16

(Fonte:https://learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/)

In [25]:
from tensorflow.keras.applications import vgg16

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(216, 216, 3))

58900480/58889256 [==============================] - 1s 0us/step


In [26]:

model = Sequential()
model.add(vgg_conv)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 6, 6, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              18875392  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 33,600,330
Trainable params: 33,600,330
Non-trainable params: 0
_________________________________________________________________


In [27]:

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_batchsize = 100
val_batchsize = 17

train_generator = train_datagen.flow_from_directory(
        '/content/data/train',
        target_size=(216, 216),
        batch_size=train_batchsize,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        '/content/data/test',
        target_size=(216, 216),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


Found 1050 images belonging to 10 classes.
Found 450 images belonging to 10 classes.


In [28]:
import tensorflow as tf

opt = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,verbose=1
)


Epoch 1/50
11/10 [===============================] - ETA: -2s - batch: 5.0000 - size: 95.4545 - loss: 2.4852 - acc: 0.1019

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10/10 [==============================] - 99s 6s/step - batch: 5.0000 - size: 95.4545 - loss: 2.4854 - acc: 0.1019 - val_loss: 2.3653 - val_acc: 0.0978
Epoch 2/50
10/10 [==============================] - 33s 3s/step - batch: 5.0000 - size: 95.4545 - loss: 2.2988 - acc: 0.1276 - val_loss: 2.3366 - val_acc: 0.1022
Epoch 3/50
10/10 [==============================] - 33s 3s/step - batch: 5.0000 - size: 95.4545 - loss: 2.2677 - acc: 0.1219 - val_loss: 2.2723 - val_acc: 0.1822
Epoch 4/50
10/10 [==============================] - 33s 3s/step - batch: 5.0000 - size: 95.4545 - loss: 2.2472 - acc: 0.1638 - val_loss: 2.3717 - val_acc: 0.1156
Epoch 5/50
10/10 [==============================] - 33s 3s/step - batch: 5.0000 - size: 95.4545 - loss: 2.1806 - acc: 0.1895 - val_loss: 2.1706 - val_acc: 0.2289
Epoch 6/50
10/10 [==============================] - 33s 3s/step - batch: 5.0000 - size: 95.4545 - loss: 2.1900 - acc: 0.1933 - val_loss: 2.2107 - val_acc: 0.1911
Epoch 7/50
10/10 [=====================

In [29]:
import matplotlib.pyplot as plt
import numpy as np

predictions = model.predict(
    validation_generator, 
    steps=validation_generator.samples/validation_generator.batch_size,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [33]:
real = []
predicoes = []
dicioVGG = {
    'bear':[],
    'bear_predicoes':[],
    'bird':[],
    'bird_predicoes':[],
    'cat':[],
    'cat_predicoes':[],
    'cow':[],
    'cow_predicoes':[],
    'dog':[],
    'dog_predicoes':[],
    'elephant':[],
    'elephant_predicoes':[],
    'giraffe':[],
    'giraffe_predicoes':[],
    'horse':[],
    'horse_predicoes':[],
    'sheep':[],
    'sheep_predicoes':[],
    'zebra':[],
    'zebra_predicoes':[]
}

def criandDict(real, predicoes):
  i=0
  while i < 450:
    dicioVGG[real[i]].append(1)
    if real[i] == predicoes[i]:
      dicioVGG[real[i]+'_predicoes'].append(1)
    else:
      dicioVGG[real[i]+'_predicoes'].append(0)
    i = i + 1

def obtain_errors(val_generator, predictions):

    fnames = validation_generator.filenames

    ground_truth = validation_generator.classes

    label2index = validation_generator.class_indices

    idx2label = list(label2index.keys())

    predicted_classes = np.argmax(predictions, axis=1)

    return idx2label, fnames, predicted_classes

def predicoesVGG(idx2label, predictions, predicted_classes,fnames):

    for i in range(len(predicted_classes)):
        
        pred_class = predicted_classes[i]

        pred_label = idx2label[pred_class]

        fnames[i].split('/')[0] #Real
        pred_label #Predicao
        #predictions[predicted_classes[i]][pred_class] #Confiança
        
        print(f"Predição: {pred_label} / Classe real: {fnames[i]}")
        

        real.append(fnames[i].split('/')[0])
        predicoes.append(pred_label)




In [34]:
idx2label, fnames, predicted_classes = obtain_errors(validation_generator, predictions)

predicoesVGG(idx2label, predictions ,predicted_classes, fnames)

criandDict(real, predicoes)

for item in dicioVGG:
  if not '_' in str(item):
    calculaAcuraciaPClasse(item, dicioVGG[item], dicioVGG[item+'_predicoes'])
    calculaf1PClasse(item, dicioVGG[item], dicioVGG[item+'_predicoes'])
    matrixConfusao(item, dicioVGG[item], dicioVGG[item+'_predicoes'])

j = 0
while j < 450:
  
  if real[j] == predicoes[j]:
    real[j] = 1
    predicoes[j] = 1
  else:
    real[j] = 1
    predicoes[j] = 0
  
  j = j + 1

print(real)
print(predicoes)
calculaAcuraciaPClasse('GERAL', real, predicoes)
calculaf1PClasse('GERAL', real, predicoes)


Predição: bear / Classe real: bear/bear_000000000285.jpg
Predição: bear / Classe real: bear/bear_000000020247.jpg
Predição: dog / Classe real: bear/bear_000000042296.jpg
Predição: sheep / Classe real: bear/bear_000000071756.jpg
Predição: sheep / Classe real: bear/bear_000000079144.jpg
Predição: bear / Classe real: bear/bear_000000085478.jpg
Predição: bear / Classe real: bear/bear_000000092839.jpg
Predição: bear / Classe real: bear/bear_000000096960.jpg
Predição: sheep / Classe real: bear/bear_000000104603.jpg
Predição: bear / Classe real: bear/bear_000000127955.jpg
Predição: bear / Classe real: bear/bear_000000132622.jpg
Predição: bear / Classe real: bear/bear_000000173033.jpg
Predição: bear / Classe real: bear/bear_000000186422.jpg
Predição: sheep / Classe real: bear/bear_000000194216.jpg
Predição: bear / Classe real: bear/bear_000000194506.jpg
Predição: bear / Classe real: bear/bear_000000203546.jpg
Predição: elephant / Classe real: bear/bear_000000205776.jpg
Predição: bear / Classe 